In [1]:
import sagemaker
from sagemaker.local import LocalSession
from argparse import Namespace
import boto3

In [2]:
args = Namespace()
args.train_instance_type = 'ml.m4.xlarge'
args.model_prefix = "dummy-whisper-async"
sess = LocalSession()
# sess = boto3.Session()
role = sagemaker.get_execution_role()
sess.config = {'local': {'local_code': True}}
# model = SKLearnModel(
#     entry_point='inference.py',
#     role=role,
#     model_data=model_artifacts,
#     framework_version='0.23-1',
# )

dummy_whisper = sagemaker.estimator.Estimator(
                                        'dummy-whisper-async',
                                        entry_point='serve',
                                        role=role,
                                        train_instance_count=1, 
                                        train_instance_type=args.train_instance_type,
    #                                     output_path='s3://{}/{}/output'.format(bucket, prefix),
    #                                       output_path='s3://sagemaker-ap-northeast-1-268945887964', 
    #                                       output_path='s3://sagemaker-ap-northeast-1-268945887964/custom_inference/',
                                          output_path=f's3://sagemaker-ap-northeast-1-268945887964/{args.model_prefix}',# this is for saving training results
                                        base_job_name=args.model_prefix,
                                        sagemaker_session=sess)
dummy_whisper.fit()

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Creating yrx2y4t3mn-algo-1-7mnu2 ... 
Creating yrx2y4t3mn-algo-1-7mnu2 ... done
Attaching to yrx2y4t3mn-algo-1-7mnu2
yrx2y4t3mn-algo-1-7mnu2 | 
yrx2y4t3mn-algo-1-7mnu2 | Starting the training.
yrx2y4t3mn-algo-1-7mnu2 | Training complete.
yrx2y4t3mn-algo-1-7mnu2 exited with code 0
Aborting on container exit...
===== Job Complete =====


In [3]:
import numpy as np
payload = np.array([[0.09178, 0.0, 4.05, 0.0, 0.51, 6.416, 84.1, 2.6463, 5.0, 296.0, 16.6, 395.5, 9.04]])
payload

array([[9.1780e-02, 0.0000e+00, 4.0500e+00, 0.0000e+00, 5.1000e-01,
        6.4160e+00, 8.4100e+01, 2.6463e+00, 5.0000e+00, 2.9600e+02,
        1.6600e+01, 3.9550e+02, 9.0400e+00]])

In [13]:
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig
# try:
if True:
#     predictor = dummy_whisper.deploy(initial_instance_count=1, instance_type='local')
    endpoint_name = f'{args.model_prefix}-endpoint'
    async_config = AsyncInferenceConfig(
     output_path = f"s3://sagemaker-ap-northeast-1-268945887964/{args.model_prefix}/response",
     max_concurrent_invocations_per_instance=1)
    predictor = dummy_whisper.deploy(initial_instance_count=1, 
                                     instance_type='local', 
                                     endpoint_name=endpoint_name,
#                                      async_inference_config1 = 2,
#                                      async_inference_config = 1,
                                     async_inference_config = async_config,
                        )
#     print(predictor)
#     preds = predictor.predict(payload)
#     print(preds)
# except Exception as e:
#     print(e)

TypeError: create_endpoint_config() got an unexpected keyword argument 'AsyncInferenceConfig'

In [14]:
%debug

> /home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py(3681)endpoint_from_production_variants()
   3679 
   3680         LOGGER.info("Creating endpoint-config with name %s", name)
-> 3681         self.sagemaker_client.create_endpoint_config(**config_options)
   3682 
   3683         return self.create_endpoint(endpoint_name=name, config_name=name, tags=tags, wait=wait)



ipdb>  config_options


{'EndpointConfigName': 'dummy-whisper-async-endpoint', 'ProductionVariants': [{'ModelName': 'dummy-whisper-async-2022-12-06-07-15-28-516', 'VariantName': 'AllTraffic', 'InitialVariantWeight': 1, 'InitialInstanceCount': 1, 'InstanceType': 'local'}], 'AsyncInferenceConfig': {'OutputConfig': {'S3OutputPath': 's3://sagemaker-ap-northeast-1-268945887964/dummy-whisper-async/response'}, 'ClientConfig': {'MaxConcurrentInvocationsPerInstance': 1}}}


ipdb>  self.sagemaker_client.create_endpoint_config


<bound method LocalSagemakerClient.create_endpoint_config of <sagemaker.local.local_session.LocalSagemakerClient object at 0x7fd2f42331c0>>


ipdb>  q


In [25]:
%%time
import json
import os
import tempfile
import base64
def send_request(audio_path,url):
    print('using',audio_path)
    #=====================================================================
    audio_base64 = base64.b64encode(open(audio_path, "rb").read())
    ext = audio_path.split('.')[-1]
    payload = audio_base64
    request_dict ={
        'data':payload.decode(),
        'ext':ext}

    with tempfile.NamedTemporaryFile(mode='w+',suffix='.json',delete=True) as f:
        json.dump(request_dict,f)    
        f.seek(0)
        
        os.system(f'bash predict.sh {url} {f.name} "application/json"')

# audio_path = '/home/ec2-user/SageMaker/Audio_James_Randi.wav'
# audio_path = '/home/ec2-user/SageMaker/Audio_James_Randi_Long_Version.ogg'
audio_path = '/home/ec2-user/SageMaker/thecantervilleghostversion2_01_wilde_128kb.mp3'
send_request(audio_path,'127.0.0.1:5000')

using /home/ec2-user/SageMaker/thecantervilleghostversion2_01_wilde_128kb.mp3


Note: Unnecessary use of -X or --request, POST is already inferred.
*   Trying 127.0.0.1:5000...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0* Connected to 127.0.0.1 (127.0.0.1) port 5000 (#0)
> POST /invocations HTTP/1.1
> Host: 127.0.0.1:5000
> User-Agent: curl/7.80.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 14196270
> Expect: 100-continue
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 100 Continue
} [65536 bytes data]
* We are completely uploaded and fine
100 13.5M    0     0  100 13.5M      0   123k  0:01:52  0:01:52 --:--:--     0

{"text":" CHAPTER I. When Mr. Hiram B. Otis, the American minister, bought Canterville Chase, everyone told him he was doing a very foolish thing, as there was no doubt at all that the place was haunted. Indeed, Lord Canterville himself, who was a man of the most punctilious honor, had felt it his duty to mention the fact to Mr. Otis when they came to discuss terms. \"'We have not cared to live in the place ourselves,' said Lord Canterville, \"'since my grandaunt, the dowager-duchess of Bolton, was frightened into a fit from which she never really recovered, by two skeleton hands being placed on her shoulders as she was dressing for dinner. And I feel bound to tell you, Mr. Otis, that the ghost has been seen by several living members of my family, as well as by the rector of the parish, the Reverend Augustus Dampier, who is a fellow of King's College, Cambridge. After the unfortunate accident to the duchess, none of our younger servants would stay with us, and Lady Canterville often go

100 13.5M    0     0  100 13.5M      0   122k  0:01:53  0:01:53 --:--:--     0* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: nginx/1.14.0 (Ubuntu)
< Date: Tue, 06 Dec 2022 15:31:04 GMT
< Content-Type: application/json
< Content-Length: 8458
< Connection: keep-alive
< 
{ [8458 bytes data]
100 13.5M  100  8458  100 13.5M     74   122k  0:01:54  0:01:53  0:00:01  2112
* Connection #0 to host 127.0.0.1 left intact


In [19]:
%cd predictor

[Errno 2] No such file or directory: 'predictor'
/home/ec2-user/SageMaker/whisper-async1/transcription


In [23]:
%cd local_test

/home/ec2-user/SageMaker/whisper-async1/local_test


In [24]:
%ls

predict.sh  send_request.py  serve_local.sh
